<a href="https://colab.research.google.com/github/robsonbfreitas/CalculadoraDeAluguel/blob/main/AnaliseExploratoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.vivareal.com.br/venda/sp/sao-paulo/apartamento_residencial/'

In [ ]:
c = requests.get(url).content

In [ ]:
soup = BeautifulSoup(c)

In [ ]:
aluguel_tag = soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'})

In [ ]:
len(aluguel_tag)
paginas = ['']
for i in range(2,40):
    a = '?pagina='+str(i)
    print(a)
    paginas.append(a)


In [ ]:
import pandas as pd

In [ ]:
def converter_df(aluguel_lista,quartos_lista,endereco_lista,banheiro_lista,vaga_lista,area_lista):
    data = {'aluguel':aluguel_lista,
           'quartos':quartos_lista,
           'endereco':endereco_lista,
           'banheiro':banheiro_lista,
            'vaga':vaga_lista,
            'area':area_lista}
    df = pd.DataFrame(data)
    return df
